In [1]:
#Import packages into python kernel
import pandas as pd
import numpy as np
import plotly.express as px
import pypyodbc
import os.path
import os,sys
from datetime import datetime, timedelta
import logging
from dotenv import load_dotenv
load_dotenv()

#Logging provide an extra set of eyes that are constantly looking at the flow and
#debug errors easily but also use the data to analyze the performance of the application 
#to plan for scaling or look at usage patterns to plan for marketing.
#Start Automation Process & save as Automation.log
logger = logging.getLogger("Automation")
logger.setLevel(logging.INFO)
if logger.hasHandlers():
    logger.handlers.clear()
file_handler = logging.FileHandler("Automation.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

logger.info("******** Automation Process *********")

In [2]:
#Get today's date into today
t = datetime.now() - timedelta(8)
today = datetime.strftime(t, '%m/%d/%Y')
today

'05/31/2022'

In [3]:
#Assign all data of Summary_Data.csv file for summary_data
summary_data = pd.read_csv(r'C:\Users\Lenovo\Summary_Data.csv')

In [4]:
#Get summary_data['f1_leg']='A', sumary_data[f1_type_name]='o-d'or'transfer'
#group by column['fl_afskey'],['fl_leg']& summation on column ['fl_type_count']
#remove column['fl_afskey']=0
a_summary = summary_data[(summary_data['fl_leg']=='A') & (summary_data['fl_type_name'].isin(['o-d','transfer']))][['fl_internal_key','fl_afskey','fl_leg','fl_type_name','fl_type_count']].reset_index(drop=True)
a_summary = a_summary.groupby(['fl_afskey','fl_leg'])['fl_type_count'].sum().reset_index()
a_summary = a_summary[a_summary['fl_afskey']!=0]

In [5]:
#Display data
a_summary.head()

,fl_afskey,fl_leg,fl_type_count
1,10162443,A,117
2,10163430,A,92
3,10180618,A,26
4,10400765,A,79
5,10427764,A,58


In [6]:
#Get summary_data['f1_leg']='D', sumary_data[f1_type_name]='o-d'or'transfer'or'transit'or'transitinfants'
#group by column['fl_afskey'],['fl_leg']& summation on column ['fl_type_count']
#remove rows for column['fl_afskey']=0

d_summary = summary_data[(summary_data['fl_leg']=='D') & (summary_data['fl_type_name'].isin(['o-d','transfer','transit','transitinfants']))][['fl_internal_key','fl_afskey','fl_leg','fl_type_name','fl_type_count']].reset_index(drop=True)
d_summary = d_summary.groupby(['fl_afskey','fl_leg'])['fl_type_count'].sum().reset_index()
d_summary = d_summary[d_summary['fl_afskey']!=0]

In [7]:
#Display data
d_summary.head()

,fl_afskey,fl_leg,fl_type_count
1,10758155,D,74
2,10758370,D,182
3,10761123,D,157
4,10762786,D,121
5,10764325,D,261


In [8]:
#Combine Dataframe of a_summary & d_summary

pd.options.display.min_rows = 100
a_d_summary = pd.concat([a_summary,d_summary]).reset_index(drop=True)
a_d_summary

,fl_afskey,fl_leg,fl_type_count
0,10162443,A,117
1,10163430,A,92
2,10180618,A,26
3,10400765,A,79
4,10427764,A,58
5,10507828,A,90
6,10645247,A,94
7,10735096,A,80
8,10748017,A,103
9,10753322,A,121


In [9]:
#Assign all data of Flight_Details.csv file for flight_details
flight_details = pd.read_csv(r'C:\Users\Lenovo\Flight_Details.csv')

In [10]:
#Assign flight_details1 equal to flight_details with columns['fl_internal_key','fl_afskey','fl_tail_registration','fl_actual_time','fl_leg','fl_aircraft_type','fl_routing']
#Remove space before & after inside the string of flight_details1['fl_routing']
#If value of column fl_leg='D' Assign flight_details1['fl_routing'] = string after coma, else flight_details1['fl_routing'] = string after coma, 
#Remove rows for column['fl_afskey']=0
flight_details1 = flight_details[['fl_internal_key','fl_afskey','fl_tail_registration','fl_actual_time','fl_leg','fl_aircraft_type','fl_routing']]
flight_details1['fl_routing'] = flight_details1['fl_routing'].str.strip()
flight_details1['fl_routing'] = flight_details1.apply(lambda val: val.fl_routing.split(",")[-1] if val.fl_leg =='D' else val.fl_routing.split(",")[0],axis=1)
flight_details1 = flight_details1[flight_details1['fl_afskey']!=0]

flight_details1

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7636\2835765229.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_details1['fl_routing'] = flight_details1['fl_routing'].str.strip()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7636\2835765229.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_details1['fl_routing'] = flight_details1.apply(lambda val: val.fl_routing.split(",")[-1] if val.fl_leg =='D' else val.fl_routing.split(",")[0],axis=1)


,fl_internal_key,fl_afskey,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing
0,3863734,10791709,A7BEN,1/03/2022 8:08,A,77W,DOH
1,3863831,10791670,9VOJJ,1/03/2022 19:27,A,789,SIN
2,3863832,10791063,9VOJJ,1/03/2022 20:48,D,789,SIN
3,3863851,10792260,HZARD,1/03/2022 12:06,A,789,MED
4,3863852,10792261,HZAK23,1/03/2022 16:57,A,77W,JED
5,3863853,10791677,TCLJF,1/03/2022 16:51,A,77W,IST
6,3863855,10791647,A6EGI,1/03/2022 19:11,A,77W,DXB
7,3863856,10791644,A6EBY,1/03/2022 14:07,A,773,DXB
8,3863857,10791652,A6EBQ,1/03/2022 23:30,A,77W,AKL
9,3863858,10791690,RPC9903,1/03/2022 10:43,A,321,MNL


In [11]:
#Merge by left join of flight_details1 as a & a_d_summary as b where a.fl_afskey=b.fl_afskey & a.fl_leg=b.fl_leg
#Remove all 0 data in dataframe of flight_details1
flight_details1 = pd.merge(flight_details1,a_d_summary,how='left',on=['fl_afskey','fl_leg'])
flight_details1 = flight_details1[flight_details1['fl_afskey']!=0]

In [12]:
#Display data
flight_details1

,fl_internal_key,fl_afskey,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,fl_type_count
0,3863734,10791709,A7BEN,1/03/2022 8:08,A,77W,DOH,187
1,3863831,10791670,9VOJJ,1/03/2022 19:27,A,789,SIN,231
2,3863832,10791063,9VOJJ,1/03/2022 20:48,D,789,SIN,291
3,3863851,10792260,HZARD,1/03/2022 12:06,A,789,MED,177
4,3863852,10792261,HZAK23,1/03/2022 16:57,A,77W,JED,360
5,3863853,10791677,TCLJF,1/03/2022 16:51,A,77W,IST,117
6,3863855,10791647,A6EGI,1/03/2022 19:11,A,77W,DXB,76
7,3863856,10791644,A6EBY,1/03/2022 14:07,A,773,DXB,257
8,3863857,10791652,A6EBQ,1/03/2022 23:30,A,77W,AKL,0
9,3863858,10791690,RPC9903,1/03/2022 10:43,A,321,MNL,9


In [13]:
#Assign all data of Active_Flights.csv file for active_flight_data
active_flight_data = pd.read_csv(r'C:\Users\Lenovo\Active_Flights.csv')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7636\3689624070.py:2: DtypeWarning: Columns (32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  active_flight_data = pd.read_csv(r'C:\Users\Lenovo\Active_Flights.csv')


In [14]:
#Remove space before & after inside the string of active_flight_data['standcode']
#Assign active_flight_data['BAY'] equal to active_flight_data['standcode']
#Assign active_flight equal to active_flight_data['modesequence'] that has 0 or 1 & 
#active_flight_data['remarkcodeflight'] that has no 'FCL' & 
#active_flight_data['tailno'] that not empty & active_flight_data['iataservicetypecode'] that has 'C' or 'G' or 'J'

active_flight_data['standcode'] = active_flight_data['standcode'].str.strip()
active_flight_data['BAY'] = active_flight_data['standcode'] 
active_flight = active_flight_data[(active_flight_data['modesequence'].isin([0,1])) & (~active_flight_data['remarkcodeflight'].isin(['FCL'])) & (~active_flight_data['tailno'].isin([''])) & (active_flight_data['iataservicetypecode'].isin(['C','G','J']))]

In [15]:
#Display data with setting of display
pd.options.display.max_rows = 20
pd.options.display.max_columns = 87
active_flight.head(20)

,afskey,flightno,leg,stad,sfskey,seasonid,officialflightdate,iataservicetypecode,flightmode,modesequence,codeshareflight,categorycode,tailno,aircrafttypecode,trafficright,aircraftversioncode,aircraftoperatorcode,pairafskey,emergencycode,mesgtype,irregularindicator,specialflightind,station1,std1,station2,sta2,std2,station3,sta3,std3,station4,sta4,std4,station5,sta5,std5,station6,sta6,endofflight,atmsetad,sitaetad,dispetad,atad,flightplanid,remarkcodeflight,flightdelaycode,checkintype,islandsalloc,schcounteropentime,schcounterclosetime,islandremark,standcode,standonchocktime,standoffchocktime,gateloungecode,prevgateloungecode,schgateopentime,schgateboardingtime,schgatefinalcalltime,schgateclosetime,remarkcodegate,useridatgate,actgateopentime,actgateboardingtime,actgatefinalcalltime,actgateclosetime,carouselcode,precarouselcode,remarkcodecarousel,useridatcarousel,carouseldelaycode,actcarouselopentime,actfirstbagtime,actlastbagtime,actcarouselclosetime,dispflag,messageid,deleteflag,memo,lastupdtime,lastupdsite,lastupduserid,lcctflightremark,mushowcheckinby,deskalloc,serviceclass,BAY
1,10493113.0,IX 0122,A,2021-05-28 18:45:00.000,NaN,W17,2021-05-28 11:00:00.000,C,0.0,0.0,NaN,I,VTAXJ,738,NaN,NaN,IX,NaN,NaN,NaN,N,N,DEL,2021-05-28 11:00:00.000,KUL,2021-05-28 18:45:00.000,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,2021-05-28 19:24:00.000,2021-05-28 18:45:00.000,2021-05-28 19:33:00.000,2021-05-28 19:26:00.000,225336.0,CLB,OPE,NaN,NaN,NaN,NaN,NaN,B006L,2021-05-28 19:34:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,CLB,NaN,NaN,2021-05-28 19:26:00.000,NaN,2021-05-28 21:13:00.000,NaN,Y,NaN,NaN,NaN,2021-05-28 21:11:01.827,FIDS,FIDFCRCM2,NaN,NaN,NaN,NaN,B006L
2,10493114.0,IX 0625,D,2021-05-28 19:45:00.000,NaN,W17,2021-05-28 19:45:00.000,C,0.0,0.0,NaN,I,VTAXJ,738,NaN,NaN,IX,NaN,NaN,NaN,N,N,KUL,2021-05-28 19:45:00.000,TRZ,2021-05-29 04:00:00.000,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,2021-05-28 21:10:00.000,2021-05-28 19:45:00.000,2021-05-28 19:45:00.000,2021-05-28 20:56:00.000,225363.0,FDP,NaN,NaN,NaN,2021-05-28 16:45:00.000,2021-05-28 19:00:00.000,NaN,B006L,NaN,2021-05-28 20:40:00.000,H006L,NaN,2021-05-28 18:45:00.000,2021-05-28 19:15:00.000,2021-05-28 19:25:00.000,2021-05-28 19:35:00.000,GCL,NaN,2021-05-28 18:23:00.000,2021-05-28 20:15:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,2021-05-28 20:56:46.243,FIDS,FIDTMATMS,NaN,NaN,NaN,NaN,B006L
22,10493149.0,MH 2743,A,2021-05-28 14:05:00.000,NaN,W17,2021-05-28 12:00:00.000,J,3.0,1.0,MH 2743,D,9MMXA,73H,NaN,NaN,MH,NaN,NaN,NaN,N,N,BTU,2021-05-28 12:00:00.000,KUL,2021-05-28 14:05:00.000,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,2021-05-28 14:38:00.000,2021-05-28 14:05:00.000,2021-05-28 14:05:00.000,2021-05-28 14:37:00.000,225278.0,CLB,NaN,NaN,NaN,NaN,NaN,NaN,B002R,2021-05-28 14:44:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L,NaN,CLB,NaN,NaN,2021-05-28 14:37:00.000,2021-05-28 14:57:00.000,2021-05-28 15:21:00.000,NaN,Y,NaN,NaN,NaN,2021-05-29 00:34:47.543,FIDS,GASSNDALC,NaN,NaN,NaN,NaN,B002R
26,10688876.0,MH 1152,D,2021-12-22 16:20:00.000,NaN,W17,2021-12-22 16:20:00.000,J,3.0,1.0,MH 1152,D,9MMLK,73H,NaN,NaN,MH,NaN,NaN,NaN,N,N,KUL,2021-12-22 16:20:00.000,PEN,2021-12-22 17:15:00.000,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,,NaN,NaN,2021-12-22 16:20:00.000,2021-12-22 16:20:00.000,2021-12-22 16:20:00.000,2021-12-22 16:25:00.000,283508.0,FDP,NaN,NaN,NaN,2021-12-22 13:20:00.000,2021-12-22 15:20:00.000,NaN,B009,NaN,2021-12-22 16:16:00.000,B009,NaN,2021-12-22 15:20:00.000,2021-12-22 15:50:00.000,2021-12-22 16:05:00.000,2021-12-22 16:10:00.000,GCL,NaN,2021-12-22 15:30:00.000,2021-12-22 15:53:00.000,2021-12-22 16:00:00.000,2021-12-22 16:10:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,2021-12-24 09:58:52.970,FIDS,GASSNDALC,NaN,NaN,NaN,NaN,B009
35,10688885.0,MH 1208,D,2021-12-22 13:20:00.000,NaN,W17,2021-12-22 13:20:00.000,J,3.0,1.0,MH 1208,D,9MMXT,73H,NaN,NaN,MH,NaN,NaN,NaN,N,N,KUL,2021-12-22 13:20:00.000,AOR,2021-12-22 14:25:00.000,NaN,,NaN,NaN,,NaN,NaN,,NaN,N

In [16]:
#Assign selected_column_1 equal to columns of active_flight [['afskey','tailno','aircrafttypecode','leg','station1','station2','station3','station4','station5','station6','iataservicetypecode','stad','atad','standcode','BAY','categorycode']]
#Copy selected_columns_1 to today_active_flight
#Rename the column of today_active_flight 'afskey' to 'fl_afskey'
selected_columns_1 = active_flight[['afskey','tailno','aircrafttypecode','leg','station1','station2','station3','station4','station5','station6','iataservicetypecode','stad','atad','standcode','BAY','categorycode']]
today_active_flight= selected_columns_1.copy().reset_index(drop=True)
today_active_flight = today_active_flight.rename(columns={'afskey':'fl_afskey'})

In [17]:
#Display data
selected_columns_1

,afskey,tailno,aircrafttypecode,leg,station1,station2,station3,station4,station5,station6,iataservicetypecode,stad,atad,standcode,BAY,categorycode
1,10493113.0,VTAXJ,738,A,DEL,KUL,,,,,C,2021-05-28 18:45:00.000,2021-05-28 19:26:00.000,B006L,B006L,I
2,10493114.0,VTAXJ,738,D,KUL,TRZ,,,,,C,2021-05-28 19:45:00.000,2021-05-28 20:56:00.000,B006L,B006L,I
22,10493149.0,9MMXA,73H,A,BTU,KUL,,,,,J,2021-05-28 14:05:00.000,2021-05-28 14:37:00.000,B002R,B002R,D
26,10688876.0,9MMLK,73H,D,KUL,PEN,,,,,J,2021-12-22 16:20:00.000,2021-12-22 16:25:00.000,B009,B009,D
35,10688885.0,9MMXT,73H,D,KUL,AOR,,,,,J,2021-12-22 13:20:00.000,2021-12-22 13:23:00.000,B011,B011,D
43,10688893.0,9MMLJ,73H,D,KUL,TGG,,,,,J,2021-12-22 15:30:00.000,2021-12-22 15:41:00.000,B004R,B004R,D
52,10688902.0,9MFFF,73H,D,KUL,KBR,,,,,J,2021-12-23 07:55:00.000,2021-12-23 08:03:00.000,B002R,B002R,D
61,10688911.0,9MMXT,73H,D,KUL,LGK,,,,,J,2021-12-22 09:00:00.000,2021-12-22 09:10:00.000,B004L,B004L,D
70,10688920.0,9MMXY,73H,D,KUL,LGK,,,,,J,2021-12-22 12:30:00.000,2021-12-22 12:39:00.000,A009,A009,D
79,10688929.0,9MMLJ,73H,D,KUL,LGK,,,,,J,2021-12-22 10:45:00.000,2021-12-22 11:03:00.000,A008R,A008R,D


In [18]:
#Display data
today_active_flight

,fl_afskey,tailno,aircrafttypecode,leg,station1,station2,station3,station4,station5,station6,iataservicetypecode,stad,atad,standcode,BAY,categorycode
0,10493113.0,VTAXJ,738,A,DEL,KUL,,,,,C,2021-05-28 18:45:00.000,2021-05-28 19:26:00.000,B006L,B006L,I
1,10493114.0,VTAXJ,738,D,KUL,TRZ,,,,,C,2021-05-28 19:45:00.000,2021-05-28 20:56:00.000,B006L,B006L,I
2,10493149.0,9MMXA,73H,A,BTU,KUL,,,,,J,2021-05-28 14:05:00.000,2021-05-28 14:37:00.000,B002R,B002R,D
3,10688876.0,9MMLK,73H,D,KUL,PEN,,,,,J,2021-12-22 16:20:00.000,2021-12-22 16:25:00.000,B009,B009,D
4,10688885.0,9MMXT,73H,D,KUL,AOR,,,,,J,2021-12-22 13:20:00.000,2021-12-22 13:23:00.000,B011,B011,D
5,10688893.0,9MMLJ,73H,D,KUL,TGG,,,,,J,2021-12-22 15:30:00.000,2021-12-22 15:41:00.000,B004R,B004R,D
6,10688902.0,9MFFF,73H,D,KUL,KBR,,,,,J,2021-12-23 07:55:00.000,2021-12-23 08:03:00.000,B002R,B002R,D
7,10688911.0,9MMXT,73H,D,KUL,LGK,,,,,J,2021-12-22 09:00:00.000,2021-12-22 09:10:00.000,B004L,B004L,D
8,10688920.0,9MMXY,73H,D,KUL,LGK,,,,,J,2021-12-22 12:30:00.000,2021-12-22 12:39:00.000,A009,A009,D
9,10688929.0,9MMLJ,73H,D,KUL,LGK,,,,,J,2021-12-22 10:45:00.000,2021-12-22 11:03:00.000,A008R,A008R,D


In [19]:
#Assign all data of Bay_Stand.csv file into Bay_standcode
Bay_standcode = pd.read_csv(r'C:\Users\Lenovo\Bay_Stand.csv')

In [20]:
#Remove space before & after inside the string of Bay_standcode['BAY']
#Assign selected_column_2 equal to columns of Bay_standcode [['BAY','Oper_Type','TERMINAL']]
#Copy selected_columns_2 to passenger_flight ?Reset Index
Bay_standcode['BAY'] = Bay_standcode['BAY'].str.strip()
selected_columns_2 = Bay_standcode[['BAY','Oper_Type','TERMINAL']]
passenger_flight = selected_columns_2.copy()

In [21]:
#Display data
passenger_flight

,BAY,Oper_Type,TERMINAL
0,A002,Passenger,KLIA
1,A002L,Passenger,KLIA
2,A002R,Passenger,KLIA
3,A003,Passenger,KLIA
4,A004,Passenger,KLIA
5,A004L,Passenger,KLIA
6,A004R,Passenger,KLIA
7,A005,Passenger,KLIA
8,A006,Passenger,KLIA
9,A006L,Passenger,KLIA


In [22]:
#Merge by left join of today_active_flight as a & passenger_flight as b where a.BAY=b.BAY
active_passenger_flight = pd.merge(today_active_flight,passenger_flight,how='left',on=['BAY'])

In [23]:
#Display data
active_passenger_flight

,fl_afskey,tailno,aircrafttypecode,leg,station1,station2,station3,station4,station5,station6,iataservicetypecode,stad,atad,standcode,BAY,categorycode,Oper_Type,TERMINAL
0,10493113.0,VTAXJ,738,A,DEL,KUL,,,,,C,2021-05-28 18:45:00.000,2021-05-28 19:26:00.000,B006L,B006L,I,Passenger,KLIA
1,10493114.0,VTAXJ,738,D,KUL,TRZ,,,,,C,2021-05-28 19:45:00.000,2021-05-28 20:56:00.000,B006L,B006L,I,Passenger,KLIA
2,10493149.0,9MMXA,73H,A,BTU,KUL,,,,,J,2021-05-28 14:05:00.000,2021-05-28 14:37:00.000,B002R,B002R,D,Passenger,KLIA
3,10688876.0,9MMLK,73H,D,KUL,PEN,,,,,J,2021-12-22 16:20:00.000,2021-12-22 16:25:00.000,B009,B009,D,Passenger,KLIA
4,10688885.0,9MMXT,73H,D,KUL,AOR,,,,,J,2021-12-22 13:20:00.000,2021-12-22 13:23:00.000,B011,B011,D,Passenger,KLIA
5,10688893.0,9MMLJ,73H,D,KUL,TGG,,,,,J,2021-12-22 15:30:00.000,2021-12-22 15:41:00.000,B004R,B004R,D,Passenger,KLIA
6,10688902.0,9MFFF,73H,D,KUL,KBR,,,,,J,2021-12-23 07:55:00.000,2021-12-23 08:03:00.000,B002R,B002R,D,Passenger,KLIA
7,10688911.0,9MMXT,73H,D,KUL,LGK,,,,,J,2021-12-22 09:00:00.000,2021-12-22 09:10:00.000,B004L,B004L,D,Passenger,KLIA
8,10688920.0,9MMXY,73H,D,KUL,LGK,,,,,J,2021-12-22 12:30:00.000,2021-12-22 12:39:00.000,A009,A009,D,Passenger,KLIA
9,10688929.0,9MMLJ,73H,D,KUL,LGK,,,,,J,2021-12-22 10:45:00.000,2021-12-22 11:03:00.000,A008R,A008R,D,Passenger,KLIA


In [24]:
#merge by left join of active_passenger_flight as a & flight_details1 as b where a.fl_afskey=b.fl_afskey
#Display data
merge_data = pd.merge(active_passenger_flight,flight_details1,how='left',on=['fl_afskey'])
merge_data

,fl_afskey,tailno,aircrafttypecode,leg,station1,station2,station3,station4,station5,station6,iataservicetypecode,stad,atad,standcode,BAY,categorycode,Oper_Type,TERMINAL,fl_internal_key,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,fl_type_count
0,10493113.0,VTAXJ,738,A,DEL,KUL,,,,,C,2021-05-28 18:45:00.000,2021-05-28 19:26:00.000,B006L,B006L,I,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10493114.0,VTAXJ,738,D,KUL,TRZ,,,,,C,2021-05-28 19:45:00.000,2021-05-28 20:56:00.000,B006L,B006L,I,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10493149.0,9MMXA,73H,A,BTU,KUL,,,,,J,2021-05-28 14:05:00.000,2021-05-28 14:37:00.000,B002R,B002R,D,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10688876.0,9MMLK,73H,D,KUL,PEN,,,,,J,2021-12-22 16:20:00.000,2021-12-22 16:25:00.000,B009,B009,D,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10688885.0,9MMXT,73H,D,KUL,AOR,,,,,J,2021-12-22 13:20:00.000,2021-12-22 13:23:00.000,B011,B011,D,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10688893.0,9MMLJ,73H,D,KUL,TGG,,,,,J,2021-12-22 15:30:00.000,2021-12-22 15:41:00.000,B004R,B004R,D,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10688902.0,9MFFF,73H,D,KUL,KBR,,,,,J,2021-12-23 07:55:00.000,2021-12-23 08:03:00.000,B002R,B002R,D,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10688911.0,9MMXT,73H,D,KUL,LGK,,,,,J,2021-12-22 09:00:00.000,2021-12-22 09:10:00.000,B004L,B004L,D,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10688920.0,9MMXY,73H,D,KUL,LGK,,,,,J,2021-12-22 12:30:00.000,2021-12-22 12:39:00.000,A009,A009,D,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10688929.0,9MMLJ,73H,D,KUL,LGK,,,,,J,2021-12-22 10:45:00.000,2021-12-22 11:03:00.000,A008R,A008R,D,Passenger,KLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#Remove NAN from column of merge_data['fl_internal_key']
#print the details of rows & columns for merge_data
merge_data = merge_data.dropna(subset=['fl_internal_key'])
print(merge_data.shape)

(17114, 25)


In [26]:
#Display data
merge_data

,fl_afskey,tailno,aircrafttypecode,leg,station1,station2,station3,station4,station5,station6,iataservicetypecode,stad,atad,standcode,BAY,categorycode,Oper_Type,TERMINAL,fl_internal_key,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,fl_type_count
4302,10869370.0,9MMXI,73H,A,KCH,KUL,,,,,J,2022-04-17 16:40:00.000,2022-04-17 16:26:00.000,A006R,A006R,D,Passenger,KLIA,3879624.0,9MMXI,17/04/2022 16:26,A,738,KCH,48.0
4304,10869372.0,PKAXY,320,A,SUB,KUL,,,,,J,2022-04-18 09:10:00.000,2022-04-18 08:48:00.000,P001,P001,I,Passenger,klia2,3879993.0,PKAXY,18/04/2022 8:48,A,320,SUB,70.0
4305,10869377.0,9MLCR,73H,D,KUL,CGK,,,,,J,2022-04-17 13:50:00.000,2022-04-17 14:11:00.000,A004L,A004L,I,Passenger,KLIA,3880728.0,9MLCR,17/04/2022 14:11,D,738,CGK,159.0
4306,10869378.0,9MLCR,73H,A,CGK,KUL,,,,,J,2022-04-17 18:50:00.000,2022-04-17 18:50:00.000,A004L,A004L,I,Passenger,KLIA,3879582.0,9MLCR,17/04/2022 18:50,A,738,CGK,91.0
4373,10869849.0,4RANC,32Q,A,CMB,KUL,,,,,J,2022-04-17 23:35:00.000,2022-04-17 23:29:00.000,C011,C011,I,Passenger,KLIA,3879723.0,4RANC,17/04/2022 23:29,A,32Q,CMB,109.0
4374,10869850.0,S2AEQ,73H,D,KUL,DAC,,,,,J,2022-04-19 06:10:00.000,2022-04-19 07:00:00.000,C011,C011,I,Passenger,KLIA,3882456.0,S2AEQ,19/04/2022 7:00,D,73H,DAC,132.0
4375,10869853.0,9VJSQ,320,D,KUL,SIN,,,,,J,2022-04-18 08:50:00.000,2022-04-18 08:45:00.000,P004R,P004R,I,Passenger,klia2,3882026.0,9VJSQ,18/04/2022 8:45,D,320,SIN,150.0
4376,10869857.0,9VJSQ,320,D,KUL,SIN,,,,,J,2022-04-18 14:00:00.000,2022-04-18 14:12:00.000,P001,P001,I,Passenger,klia2,3881561.0,9VJSQ,18/04/2022 14:12,D,320,SIN,161.0
4377,10869863.0,9MVAA,32Q,D,KUL,SIN,,,,,J,2022-04-18 10:05:00.000,2022-04-18 10:07:00.000,P003,P003,I,Passenger,klia2,3880610.0,9MVAA,18/04/2022 10:07,D,32Q,SIN,140.0
4378,10869864.0,9MRAC,32A,D,KUL,SBW,,,,,J,2022-04-18 12:10:00.000,2022-04-18 12:26:00.000,J005,J005,D,Passenger,klia2,3880642.0,9MRAC,18/04/2022 12:26,D,32N,SBW,127.0


In [27]:
#Write all data of merge_data to csv file with name of merge_data_post_covid_IATA
merge_data.to_csv('merge_data_post_covid_IATA.csv',index=False)

In [28]:
#Assign flight_details2 equal to columns of merge_data [['fl_afskey','fl_internal_key','fl_tail_registration','fl_actual_time','fl_leg','fl_aircraft_type','fl_routing','categorycode','TERMINAL','fl_type_count']]
#print the details of rows & columns for flight_details2
flight_details2 = merge_data[['fl_afskey','fl_internal_key','fl_tail_registration','fl_actual_time','fl_leg','fl_aircraft_type','fl_routing','categorycode','TERMINAL','fl_type_count']]
print(flight_details2.shape)

(17114, 10)


In [29]:
#Assign unique_combo equal to columns of flight_details2['fl_tail_registration','fl_leg','fl_routing'] 
#by grouping on counting flight_details2[fl_afskey]
unique_combo = flight_details2.groupby(['fl_tail_registration','fl_leg','fl_routing'])['fl_afskey'].count().reset_index()

In [30]:
#Display data
unique_combo

,fl_tail_registration,fl_leg,fl_routing,fl_afskey
0,4RABM,A,CMB,2
1,4RABM,D,CMB,2
2,4RABN,A,CMB,8
3,4RABN,D,CMB,8
4,4RABO,A,CMB,11
5,4RABO,D,CMB,11
6,4RABQ,A,CMB,3
7,4RABQ,D,CMB,3
8,4RALA,A,CMB,1
9,4RALA,D,CMB,1


In [31]:
#Assign unique_combo['Available'] equal to 'Yes'
#Display data
unique_combo['Available'] = 'Yes'
unique_combo

,fl_tail_registration,fl_leg,fl_routing,fl_afskey,Available
0,4RABM,A,CMB,2,Yes
1,4RABM,D,CMB,2,Yes
2,4RABN,A,CMB,8,Yes
3,4RABN,D,CMB,8,Yes
4,4RABO,A,CMB,11,Yes
5,4RABO,D,CMB,11,Yes
6,4RABQ,A,CMB,3,Yes
7,4RABQ,D,CMB,3,Yes
8,4RALA,A,CMB,1,Yes
9,4RALA,D,CMB,1,Yes


In [32]:
#Write all data of merge_data to csv file with name of Unique_combo_post_covid_IATA without writing index
#print the details of rows & columns for
unique_combo.to_csv('Unique_combo_post_covid_IATA.csv',index=False)
print(unique_combo.shape)

(4316, 5)


In [33]:
#Assign for_new_combo equal to columns of flight_details2['fl_leg','fl_routing']
#by grouping on calculate mean of flight_details2['fl_type_count']
for_new_combo = flight_details2.groupby(['fl_leg','fl_routing'])['fl_type_count'].mean().reset_index()
print(for_new_combo.shape)

(146, 3)


In [34]:
#Write all data of merge_data to csv file with name of for_new_combo_post_covid_IATA
for_new_combo.to_csv('for_new_combo_post_covid_IATA.csv',index=False)

In [35]:
#Write all data of merge_data to csv file with name of flight_level_pax_count2_post_covid_IATA
flight_details2.to_csv('flight_level_pax_count2_post_covid_IATA.csv',index=False)

In [36]:
#Assign all data of flight_level_pax_count2_post_covid_IATA.csv to data
data = pd.read_csv('flight_level_pax_count2_post_covid_IATA.csv')

In [37]:
#Display data
data

,fl_afskey,fl_internal_key,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,fl_type_count
0,10869370.0,3879624.0,9MMXI,17/04/2022 16:26,A,738,KCH,D,KLIA,48.0
1,10869372.0,3879993.0,PKAXY,18/04/2022 8:48,A,320,SUB,I,klia2,70.0
2,10869377.0,3880728.0,9MLCR,17/04/2022 14:11,D,738,CGK,I,KLIA,159.0
3,10869378.0,3879582.0,9MLCR,17/04/2022 18:50,A,738,CGK,I,KLIA,91.0
4,10869849.0,3879723.0,4RANC,17/04/2022 23:29,A,32Q,CMB,I,KLIA,109.0
5,10869850.0,3882456.0,S2AEQ,19/04/2022 7:00,D,73H,DAC,I,KLIA,132.0
6,10869853.0,3882026.0,9VJSQ,18/04/2022 8:45,D,320,SIN,I,klia2,150.0
7,10869857.0,3881561.0,9VJSQ,18/04/2022 14:12,D,320,SIN,I,klia2,161.0
8,10869863.0,3880610.0,9MVAA,18/04/2022 10:07,D,32Q,SIN,I,klia2,140.0
9,10869864.0,3880642.0,9MRAC,18/04/2022 12:26,D,32N,SBW,D,klia2,127.0


In [38]:
#Convert pandas string object into pandas datetime object,
#if invalid parsing, the value will be set as NaT(Not-a-Time, indicate an unknown or missing datetime value)
#Display
data['fl_actual_time'] = pd.to_datetime(data['fl_actual_time'],errors='coerce')
data['fl_actual_time']

0       2022-04-17 16:26:00
1       2022-04-18 08:48:00
2       2022-04-17 14:11:00
3       2022-04-17 18:50:00
4       2022-04-17 23:29:00
5       2022-04-19 07:00:00
6       2022-04-18 08:45:00
7       2022-04-18 14:12:00
8       2022-04-18 10:07:00
9       2022-04-18 12:26:00
                ...        
17104   2022-04-17 13:54:00
17105   2022-04-17 20:58:00
17106   2022-04-17 16:55:00
17107   2022-04-17 18:12:00
17108   2022-04-17 19:04:00
17109   2022-04-17 18:36:00
17110   2022-04-17 18:41:00
17111   2022-04-17 14:30:00
17112   2022-04-18 07:07:00
17113   2022-04-18 07:20:00
Name: fl_actual_time, Length: 17114, dtype: datetime64[ns]

In [39]:
#Use function lamda to convert datetime object to object
data['fl_actual_time'] = data['fl_actual_time'].apply(lambda val : val.strftime('%Y-%m-%d %H:%M:%S'))

In [40]:
#Display data
data['fl_actual_time']

0        2022-04-17 16:26:00
1        2022-04-18 08:48:00
2        2022-04-17 14:11:00
3        2022-04-17 18:50:00
4        2022-04-17 23:29:00
5        2022-04-19 07:00:00
6        2022-04-18 08:45:00
7        2022-04-18 14:12:00
8        2022-04-18 10:07:00
9        2022-04-18 12:26:00
                ...         
17104    2022-04-17 13:54:00
17105    2022-04-17 20:58:00
17106    2022-04-17 16:55:00
17107    2022-04-17 18:12:00
17108    2022-04-17 19:04:00
17109    2022-04-17 18:36:00
17110    2022-04-17 18:41:00
17111    2022-04-17 14:30:00
17112    2022-04-18 07:07:00
17113    2022-04-18 07:20:00
Name: fl_actual_time, Length: 17114, dtype: object

In [41]:
#Convert string into pandas datetime object
#Display data
data['fl_actual_time'] = pd.to_datetime(data['fl_actual_time'],errors='coerce')
data['fl_actual_time']

0       2022-04-17 16:26:00
1       2022-04-18 08:48:00
2       2022-04-17 14:11:00
3       2022-04-17 18:50:00
4       2022-04-17 23:29:00
5       2022-04-19 07:00:00
6       2022-04-18 08:45:00
7       2022-04-18 14:12:00
8       2022-04-18 10:07:00
9       2022-04-18 12:26:00
                ...        
17104   2022-04-17 13:54:00
17105   2022-04-17 20:58:00
17106   2022-04-17 16:55:00
17107   2022-04-17 18:12:00
17108   2022-04-17 19:04:00
17109   2022-04-17 18:36:00
17110   2022-04-17 18:41:00
17111   2022-04-17 14:30:00
17112   2022-04-18 07:07:00
17113   2022-04-18 07:20:00
Name: fl_actual_time, Length: 17114, dtype: datetime64[ns]

In [42]:
#Extract hour value of data['fl_actual_time'] to data['hour']
#Display data
data['hour'] = data['fl_actual_time'].dt.hour
data['hour']

0        16
1         8
2        14
3        18
4        23
5         7
6         8
7        14
8        10
9        12
         ..
17104    13
17105    20
17106    16
17107    18
17108    19
17109    18
17110    18
17111    14
17112     7
17113     7
Name: hour, Length: 17114, dtype: int64

In [43]:
#Assign data['shift_time'] by condition to values 
#whereby if hour<8, data['shift_time']='0000 to 0800'
#else 15>hour>=8, data['shift_time']='0800 to 1500'
#else 21>hour>=15, data['shift_time']='1500 to 2100',
#else hour>=21, data['shift_time']='2100 to 2400''
conditions = [
    (data['hour'] < 8),
    (data['hour'] >= 8) & (data['hour'] < 15),
    (data['hour'] >= 15) & (data['hour'] < 21),
    (data['hour'] >= 21)
    ]

values = ['0000 to 0800', '0800 to 1500', '1500 to 2100', '2100 to 2400']

data['shift_time'] = np.select(conditions, values)
data['shift_time']

0        1500 to 2100
1        0800 to 1500
2        0800 to 1500
3        1500 to 2100
4        2100 to 2400
5        0000 to 0800
6        0800 to 1500
7        0800 to 1500
8        0800 to 1500
9        0800 to 1500
             ...     
17104    0800 to 1500
17105    1500 to 2100
17106    1500 to 2100
17107    1500 to 2100
17108    1500 to 2100
17109    1500 to 2100
17110    1500 to 2100
17111    0800 to 1500
17112    0000 to 0800
17113    0000 to 0800
Name: shift_time, Length: 17114, dtype: object

In [44]:
#Convert string value of data['fl_actual_time'] into datetime object
#Apply function to extract value of year, month & date into data['fl_actual_time'] become obejct
#Assign grp_df equal column of data['fl_actual_time'] by grouping on summing of data['fl_actual_time']
#Convert string value of data['fl_actual_time'] into datetime object
data['fl_actual_time'] = pd.to_datetime(data['fl_actual_time'],errors='coerce')
data['fl_actual_time'] = data['fl_actual_time'].apply(lambda val : val.strftime('%Y-%m-%d'))

grp_df = data.groupby(['fl_actual_time'])['fl_type_count'].sum().reset_index()

data['fl_actual_time'] = pd.to_datetime(data['fl_actual_time'],errors='coerce')

In [45]:
#Display data
grp_df

,fl_actual_time,fl_type_count
0,2022-01-03,37385.0
1,2022-01-04,47455.0
2,2022-02-03,37438.0
3,2022-02-04,41517.0
4,2022-03-03,40215.0
5,2022-03-04,37219.0
6,2022-03-13,44566.0
7,2022-03-14,38856.0
8,2022-03-15,37792.0
9,2022-03-16,38333.0


In [46]:
#Print columns & rows details for data
#Print number of unique values for column of data[fl_routing]
#Print number of unique values for column of data[fl_aircraft_type]
print(data.shape)
print(data.fl_routing.nunique())
print(data.fl_aircraft_type.nunique())

(17114, 12)
75
35


In [47]:
#Assigning data column for year, month, week, day & day of week
data['year'] = data['fl_actual_time'].dt.year
data['month'] = data['fl_actual_time'].dt.month
data['week'] = data['fl_actual_time'].dt.week
data['day'] = data['fl_actual_time'].dt.day
data['dayofweek'] = data['fl_actual_time'].dt.dayofweek

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7636\2378679227.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['week'] = data['fl_actual_time'].dt.week


In [48]:
#Define function to get week number in a month.
#with input parameters: dates (pd.Series): Series of dates.
#Output returns: pd.Series: Week number in a month.
#pd.to_timedelta(dates.dt.day - 1, unit='D') = Convert into timedelta in unit of days
#.dt.weekday = Extrat weekday value, eg monday=0 & sunday=6
def weekinmonth(dates):
    firstday_in_month = dates - pd.to_timedelta(dates.dt.day - 1, unit='d')
    return (dates.dt.day-1 + firstday_in_month.dt.weekday) // 7 + 1

In [49]:
#Execute weekinmonth funtion to get output value of week's number in the month
data['WOM'] = weekinmonth(data['fl_actual_time'])

In [50]:
#Display data
data

,fl_afskey,fl_internal_key,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,fl_type_count,hour,shift_time,year,month,week,day,dayofweek,WOM
0,10869370.0,3879624.0,9MMXI,2022-04-17,A,738,KCH,D,KLIA,48.0,16,1500 to 2100,2022,4,15,17,6,3
1,10869372.0,3879993.0,PKAXY,2022-04-18,A,320,SUB,I,klia2,70.0,8,0800 to 1500,2022,4,16,18,0,4
2,10869377.0,3880728.0,9MLCR,2022-04-17,D,738,CGK,I,KLIA,159.0,14,0800 to 1500,2022,4,15,17,6,3
3,10869378.0,3879582.0,9MLCR,2022-04-17,A,738,CGK,I,KLIA,91.0,18,1500 to 2100,2022,4,15,17,6,3
4,10869849.0,3879723.0,4RANC,2022-04-17,A,32Q,CMB,I,KLIA,109.0,23,2100 to 2400,2022,4,15,17,6,3
5,10869850.0,3882456.0,S2AEQ,2022-04-19,D,73H,DAC,I,KLIA,132.0,7,0000 to 0800,2022,4,16,19,1,4
6,10869853.0,3882026.0,9VJSQ,2022-04-18,D,320,SIN,I,klia2,150.0,8,0800 to 1500,2022,4,16,18,0,4
7,10869857.0,3881561.0,9VJSQ,2022-04-18,D,320,SIN,I,klia2,161.0,14,0800 to 1500,2022,4,16,18,0,4
8,10869863.0,3880610.0,9MVAA,2022-04-18,D,32Q,SIN,I,klia2,140.0,10,0800 to 1500,2022,4,16,18,0,4
9,10869864.0,3880642.0,9MRAC,2022-04-18,D,32N,SBW,D,klia2,127.0,12,0800 to 1500,2022,4,16,18,0,4


In [51]:
#Assign sorted values of data['fl_actual_time'] into temp
temp = data.sort_values('fl_actual_time')

In [52]:
#Display data with setting
pd.options.display.max_rows= 20
temp.head(20)

,fl_afskey,fl_internal_key,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,fl_type_count,hour,shift_time,year,month,week,day,dayofweek,WOM
2702,10791898.0,3864166.0,9MMSA,2022-01-03,A,738,BKI,D,KLIA,58.0,22,2100 to 2400,2022,1,1,3,0,2
2722,10791985.0,3863902.0,9MMTC,2022-01-03,A,333,SIN,I,KLIA,159.0,15,1500 to 2100,2022,1,1,3,0,2
2721,10791984.0,3865770.0,9MMLN,2022-01-03,A,738,KBR,D,KLIA,126.0,16,1500 to 2100,2022,1,1,3,0,2
2720,10791983.0,3864906.0,9MRCH,2022-01-03,A,320,LGK,D,klia2,96.0,11,0800 to 1500,2022,1,1,3,0,2
2719,10791982.0,3864916.0,9MVAB,2022-01-03,A,32Q,SIN,I,klia2,158.0,12,0800 to 1500,2022,1,1,3,0,2
2718,10791967.0,3864887.0,9MAFF,2022-01-03,A,320,LGK,D,klia2,71.0,11,0800 to 1500,2022,1,1,3,0,2
2717,10791966.0,3865042.0,9MRAR,2022-01-03,A,32N,TWU,D,klia2,147.0,19,1500 to 2100,2022,1,1,3,0,2
2716,10791965.0,3864169.0,9MMTY,2022-01-03,A,332,JED,I,KLIA,243.0,16,1500 to 2100,2022,1,1,3,0,2
2715,10791956.0,3864150.0,9MMXU,2022-01-03,A,73H,KTM,I,KLIA,116.0,19,1500 to 2100,2022,1,1,3,0,2
2714,10791955.0,3864897.0,9MRAO,2022-01-03,A,32N,SBW,D,klia2,145.0,11,0800 to 1500,2022,1,1,3,0,2


In [53]:
#Grouping columns of temp['fl_tail_registration','fl_routing','fl_leg','categorycode','TERMINAL'] by 'mean', 'min', 'max', 'median', 'last'
temp = temp.groupby(['fl_tail_registration','fl_routing','fl_leg','categorycode','TERMINAL'])['fl_type_count'].agg(['mean','min','max','median','last']).reset_index()

In [54]:
#Display data
temp

,fl_tail_registration,fl_routing,fl_leg,categorycode,TERMINAL,mean,min,max,median,last
0,4RABM,CMB,A,I,KLIA,101.000000,89.0,113.0,101.0,113.0
1,4RABM,CMB,D,I,KLIA,119.500000,115.0,124.0,119.5,124.0
2,4RABN,CMB,A,I,KLIA,116.125000,93.0,128.0,118.5,118.0
3,4RABN,CMB,D,I,KLIA,117.625000,103.0,125.0,118.5,124.0
4,4RABO,CMB,A,I,KLIA,107.636364,30.0,132.0,117.0,131.0
5,4RABO,CMB,D,I,KLIA,109.545455,66.0,131.0,117.0,127.0
6,4RABQ,CMB,A,I,KLIA,107.000000,81.0,154.0,86.0,81.0
7,4RABQ,CMB,D,I,KLIA,89.333333,70.0,114.0,84.0,70.0
8,4RALA,CMB,A,I,KLIA,132.000000,132.0,132.0,132.0,132.0
9,4RALA,CMB,D,I,KLIA,122.000000,122.0,122.0,122.0,122.0


In [55]:
temp1 = pd.merge(data,temp,how='left',on=['fl_tail_registration','fl_routing','fl_leg','categorycode','TERMINAL'])

In [56]:
temp1 = temp1.drop(['fl_afskey','fl_aircraft_type','fl_actual_time','fl_aircraft_type','hour','shift_time','year','month','week','day','dayofweek','WOM'], axis = 1)

In [57]:
temp1

,fl_internal_key,fl_tail_registration,fl_leg,fl_routing,categorycode,TERMINAL,fl_type_count,mean,min,max,median,last
0,3879624.0,9MMXI,A,KCH,D,KLIA,48.0,126.125000,48.0,159.0,131.0,131.0
1,3879993.0,PKAXY,A,SUB,I,klia2,70.0,67.461538,40.0,104.0,69.0,40.0
2,3880728.0,9MLCR,D,CGK,I,KLIA,159.0,145.687500,117.0,162.0,149.5,150.0
3,3879582.0,9MLCR,A,CGK,I,KLIA,91.0,74.187500,17.0,145.0,74.0,77.0
4,3879723.0,4RANC,A,CMB,I,KLIA,109.0,111.222222,29.0,165.0,118.0,142.0
5,3882456.0,S2AEQ,D,DAC,I,KLIA,132.0,110.625000,69.0,144.0,111.5,94.0
6,3882026.0,9VJSQ,D,SIN,I,klia2,150.0,96.052632,8.0,161.0,107.0,100.0
7,3881561.0,9VJSQ,D,SIN,I,klia2,161.0,96.052632,8.0,161.0,107.0,100.0
8,3880610.0,9MVAA,D,SIN,I,klia2,140.0,161.352941,118.0,203.0,166.0,125.0
9,3880642.0,9MRAC,D,SBW,D,klia2,127.0,101.777778,66.0,131.0,100.0,120.0


In [58]:
#Write all data of temp into pax_agg_post_covid_IATA.csv whereby do not re-indexing
temp1.to_csv('pax_agg_post_covid_IATA.csv',index=False)

In [59]:
#Merging dataframe of data & temp by left join of data as a & temp as b,
#whereby a.fl_tail_registration = b.fl_tail_registration, a.fl_routing = b.fl_routing, a.fl_leg =b.fl_leg,
#a.categorycode = b.categorycode & a.TERMINAL = b.TERMINAL
data = pd.merge(data,temp,how='left',on=['fl_tail_registration','fl_routing','fl_leg','categorycode','TERMINAL'])

In [60]:
data

,fl_afskey,fl_internal_key,fl_tail_registration,fl_actual_time,fl_leg,fl_aircraft_type,fl_routing,categorycode,TERMINAL,fl_type_count,hour,shift_time,year,month,week,day,dayofweek,WOM,mean,min,max,median,last
0,10869370.0,3879624.0,9MMXI,2022-04-17,A,738,KCH,D,KLIA,48.0,16,1500 to 2100,2022,4,15,17,6,3,126.125000,48.0,159.0,131.0,131.0
1,10869372.0,3879993.0,PKAXY,2022-04-18,A,320,SUB,I,klia2,70.0,8,0800 to 1500,2022,4,16,18,0,4,67.461538,40.0,104.0,69.0,40.0
2,10869377.0,3880728.0,9MLCR,2022-04-17,D,738,CGK,I,KLIA,159.0,14,0800 to 1500,2022,4,15,17,6,3,145.687500,117.0,162.0,149.5,150.0
3,10869378.0,3879582.0,9MLCR,2022-04-17,A,738,CGK,I,KLIA,91.0,18,1500 to 2100,2022,4,15,17,6,3,74.187500,17.0,145.0,74.0,77.0
4,10869849.0,3879723.0,4RANC,2022-04-17,A,32Q,CMB,I,KLIA,109.0,23,2100 to 2400,2022,4,15,17,6,3,111.222222,29.0,165.0,118.0,142.0
5,10869850.0,3882456.0,S2AEQ,2022-04-19,D,73H,DAC,I,KLIA,132.0,7,0000 to 0800,2022,4,16,19,1,4,110.625000,69.0,144.0,111.5,94.0
6,10869853.0,3882026.0,9VJSQ,2022-04-18,D,320,SIN,I,klia2,150.0,8,0800 to 1500,2022,4,16,18,0,4,96.052632,8.0,161.0,107.0,100.0
7,10869857.0,3881561.0,9VJSQ,2022-04-18,D,320,SIN,I,klia2,161.0,14,0800 to 1500,2022,4,16,18,0,4,96.052632,8.0,161.0,107.0,100.0
8,10869863.0,3880610.0,9MVAA,2022-04-18,D,32Q,SIN,I,klia2,140.0,10,0800 to 1500,2022,4,16,18,0,4,161.352941,118.0,203.0,166.0,125.0
9,10869864.0,3880642.0,9MRAC,2022-04-18,D,32N,SBW,D,klia2,127.0,12,0800 to 1500,2022,4,16,18,0,4,101.777778,66.0,131.0,100.0,120.0


In [61]:
#Model Building
from pycaret.regression import *

exp_1 = setup(data=data,target='fl_type_count',train_size=0.8,
              categorical_features=['fl_tail_registration','fl_leg','fl_routing','categorycode','TERMINAL','shift_time'],
              ignore_features=['fl_actual_time','fl_afskey','fl_internal_key','fl_aircraft_type'],
              numeric_features=['year','month','week','day','dayofweek','WOM','hour','mean','median','min','max','last'],session_id=123,silent=True
             )

C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sktime\utils\validation\series.py:21: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sktime\forecasting\base\_fh.py:18: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  RELATIVE_TYPES = (pd.Int64Index, pd.RangeIndex)
C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sktime\forecasting\base\_fh.py:19: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  ABSOLUTE_TYPES = (pd.Int64Index, pd.RangeIndex, pd.DatetimeIndex, pd.PeriodIndex)
C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sktime\datat

,Description,Value
0,Session id,123
1,Target,fl_type_count
2,Target type,regression
3,Data shape,"(17114, 19)"
4,Train data shape,"(13691, 19)"
5,Test data shape,"(3423, 19)"
6,Ignore features,4
7,Ordinal features,2
8,Numeric features,12
9,Categorical features,6


In [62]:
#Comparing all ML models
best_models = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,20.0790,779.5138,27.9062,0.6956,0.3266,0.2485,1.5390
ridge,Ridge Regression,20.0749,779.4045,27.9044,0.6956,0.3267,0.2485,0.2380
br,Bayesian Ridge,20.0497,779.8309,27.9124,0.6954,0.3268,0.2484,0.2530
et,Extra Trees Regressor,19.2017,785.6028,28.0095,0.6934,0.3142,0.2345,1.7340
en,Elastic Net,20.0609,785.8800,28.0209,0.6931,0.3262,0.2485,0.2630
lasso,Lasso Regression,20.0627,786.0581,28.0240,0.6930,0.3260,0.2484,0.2870
omp,Orthogonal Matching Pursuit,20.0463,787.6197,28.0510,0.6924,0.3198,0.2463,0.2520
gbr,Gradient Boosting Regressor,20.5233,809.5947,28.4365,0.6840,0.3437,0.2642,0.9740
huber,Huber Regressor,19.0669,857.0540,29.2534,0.6655,0.3240,0.2355,0.4250
lightgbm,Light Gradient Boosting Machine,21.4183,898.4583,29.9591,0.6494,0.3484,0.2770,0.3340


In [63]:
#List ML Models
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [64]:
#Create Linear Regression Model
lr = create_model('lr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,19.7373,717.7037,26.7900,0.7233,0.3037,0.2468
1,19.9535,762.1155,27.6064,0.7279,0.3301,0.2211
2,19.7596,747.2069,27.3351,0.6973,0.3252,0.2427
3,20.3048,807.7239,28.4205,0.6916,0.3459,0.2599
4,20.3937,817.8257,28.5977,0.6784,0.3316,0.2667
5,19.7005,740.5821,27.2136,0.6756,0.3255,0.2117
6,20.1923,767.7860,27.7090,0.7020,0.3317,0.2584
7,20.4652,819.1047,28.6200,0.6896,0.3680,0.2800
8,19.8449,730.7513,27.0324,0.7122,0.2874,0.2426
9,20.4387,884.3385,29.7378,0.6577,0.3166,0.2551


In [65]:
#Tune model by automatic choosing better
tuned_lr = tune_model(lr, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,19.7373,717.7037,26.7900,0.7233,0.3037,0.2468
1,19.9535,762.1155,27.6064,0.7279,0.3301,0.2211
2,19.7596,747.2069,27.3351,0.6973,0.3252,0.2427
3,20.3048,807.7239,28.4205,0.6916,0.3459,0.2599
4,20.3937,817.8257,28.5977,0.6784,0.3316,0.2667
5,19.7005,740.5821,27.2136,0.6756,0.3255,0.2117
6,20.1923,767.7860,27.7090,0.7020,0.3317,0.2584
7,20.4652,819.1047,28.6200,0.6896,0.3680,0.2800
8,19.8449,730.7513,27.0324,0.7122,0.2874,0.2426
9,20.4387,884.3385,29.7378,0.6577,0.3166,0.2551


In [66]:
#Tune model optuna
tune_model(lr, search_library = 'optuna')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,19.7373,717.7037,26.7900,0.7233,0.3037,0.2468
1,19.9535,762.1155,27.6064,0.7279,0.3301,0.2211
2,19.7596,747.2069,27.3351,0.6973,0.3252,0.2427
3,20.3048,807.7239,28.4205,0.6916,0.3459,0.2599
4,20.3937,817.8257,28.5977,0.6784,0.3316,0.2667
5,19.7005,740.5821,27.2136,0.6756,0.3255,0.2117
6,20.1923,767.7860,27.7090,0.7020,0.3317,0.2584
7,20.4652,819.1047,28.6200,0.6896,0.3680,0.2800
8,19.8449,730.7513,27.0324,0.7122,0.2874,0.2426
9,20.4387,884.3385,29.7378,0.6577,0.3166,0.2551


LinearRegression(n_jobs=-1)

In [67]:
#Tune model scikit-optimize
tune_model(lr, search_library = 'scikit-optimize')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,19.7373,717.7037,26.7900,0.7233,0.3037,0.2468
1,19.9535,762.1155,27.6064,0.7279,0.3301,0.2211
2,19.7596,747.2069,27.3351,0.6973,0.3252,0.2427
3,20.3048,807.7239,28.4205,0.6916,0.3459,0.2599
4,20.3937,817.8257,28.5977,0.6784,0.3316,0.2667
5,19.7005,740.5821,27.2136,0.6756,0.3255,0.2117
6,20.1923,767.7860,27.7090,0.7020,0.3317,0.2584
7,20.4652,819.1047,28.6200,0.6896,0.3680,0.2800
8,19.8449,730.7513,27.0324,0.7122,0.2874,0.2426
9,20.4387,884.3385,29.7378,0.6577,0.3166,0.2551


LinearRegression(n_jobs=-1)

In [ ]:
#Tune model tune-sklearn
tune_model(lr, search_library = 'tune-sklearn', search_algorithm = 'hyperopt')

2022-06-08 19:24:03,062	INFO trial_runner.py:803 -- starting _Trainable_800f739c
2022-06-08 19:24:03,124	INFO trial_runner.py:803 -- starting _Trainable_801d94fe
2022-06-08 19:24:03,165	INFO trial_runner.py:803 -- starting _Trainable_802225c4
2022-06-08 19:24:03,209	INFO trial_runner.py:803 -- starting _Trainable_8028b102
2022-06-08 19:24:03,265	INFO trial_runner.py:803 -- starting _Trainable_802f8a16
2022-06-08 19:24:03,319	INFO trial_runner.py:803 -- starting _Trainable_8038f97f
(_Trainable pid=13456) C:\Users\Lenovo\.conda\envs\tpe\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
(_Trainable pid=13456)   warnings.warn(
(_Trainable pid=13752) C:\Users\Lenovo\.con

In [ ]:
#Plot tuned Linear Regression Model residual plot
plot_model(tuned_lr)

In [ ]:
#Plot tuned Linear Regression Model prediction error plot
plot_model(tuned_lr, plot = 'error')

In [ ]:
#Plot tuned Linear Regression Model feature important plot
plot_model(tuned_lr, plot='feature')

In [ ]:
#Analyze performance of tuned Linear Regression Model
evaluate_model(tuned_lr)

In [ ]:
#Predict On Test Sample
predict_model(tuned_lr);

In [ ]:
#Finalize tuned Extra Trees Regressor Model for deployment
final_lr = finalize_model(tuned_lr)

In [ ]:
#Deploy
predict_model(final_lr);

In [ ]:
#Save tuned Extra Trees Regressor Model
save_model(final_lr,'Final_Linear_Regression')

In [ ]:
pd.options.display.max_columns = 23
data

In [ ]:
data.shape